From https://medium.com/opex-analytics/optimization-modeling-in-python-pulp-gurobi-and-cplex-83a62129807a:

We seek to solve the following for $x_{ij}$:
\begin{align*}
&\min\left\{\sum_{i\in I}\sum_{j\in J}c_{ij}x_{ij}\right\},\\
&\sum_{i\in I}a_{ij}x_{ij}\leq b_j \ \ (\textrm{Constraint})\\
&l_{ij}\leq x_{ij}\leq u_{ij}
\end{align*}
for all $i,j$.

Another good reference is the article written here: 

https://pythonpool.com/cplex-python/

In [4]:
#import libraries
import random
import gurobipy as grb
import docplex.mp.model as  cpx
import pulp as plp
import operator as op
import pandas as pd
from cplex import *
from docplex import *

In [5]:
#changeable parameter values to fit the problem
n=2
m=1
c_start = 0
c_end =100
a_start = 0
a_end = 1
l_start = 0
l_end = 100
u_start = 100
u_end = 150
b_start = 0
b_end = 100
x_vars_type = "Continuous" #The x variable will be continuous ("Continuous"), binary ("Binary"), or integer valued ("Integer").
opt_model_type = "CPX"
name_format = "x_{0}_{1}" #decision variables
constraint_name = "constraint_{0}"
constraint_type = "leq" #values will be leq, req, eq
objective_type = "min" #values will be max or min

In [6]:
set_I = range(1,n+1)
set_J = range(1,m+1)
c_range = random.normalvariate(c_start,c_end)/100
a_range =random.normalvariate(a_start,a_end)/100
l_range = random.randint(l_start,l_end)/100
u_range = random.randint(u_start,u_end)/100
b_range = random.randint(b_start,b_end)/100
# we assume variabels c, a, l, u, and b are given to us as a part of the modeling process
c = {(i,j): random.normalvariate(c_start,c_end) for i in set_I for j in set_J}
a = {(i,j): random.normalvariate(a_start,a_end)/100 for i in set_I for j in set_J}
lower_bound = {(i,j): random.randint(l_start,l_end)/100 for i in set_I for j in set_J}
upper_bound = {(i,j): random.randint(u_start,u_end)/100 for i in set_I for j in set_J}
b = {j: random.randint(b_start,b_end)/100 for j in set_J}

In [8]:
c

{(1, 1): 78.54528785547316, (2, 1): -129.9591692832855}

### Set the opt_model_type and v_type

In [5]:
if x_vars_type == "Continuous":
    if opt_model_type == "GRB":
        opt_model = grb.Model(name="MIP Model")
        var_type = grb.GRB.CONTINUOUS
    elif opt_model_type == "CPX":
        opt_model = cpx.Model(name="MIP Model")
        var_type = opt_model.continuous_var
    elif opt_model_type == "PLP":
        opt_model = plp.LpProblem(name="MIP Model")
        var_type = plp.LpContinuous
    else:
        print('Incorrect opt_model_type. Must be grb, cpx, or plp.')
elif x_vars_type == "Binary":
    if opt_model_type == "GRB":
        opt_model = grb.Model(name="MIP Model")
        var_type = grb.GRB.BINARY
    elif opt_model_type == "CPX":
        opt_model = cpx.Model(name="MIP Model")
        var_type = opt_model.binary_var
    elif opt_model_type == "PLP":
        opt_model = plp.LpProblem(name="MIP Model")
        var_type = plp.LpBinary
    else:
        print('Incorrect opt_model_type. Must be grb, cpx, or plp.') 
elif x_vars_type == "Integer":
    if opt_model_type == "GRB":
        opt_model = grb.Model(name="MIP Model")
        var_type = grb.GRB.INTEGER
    elif opt_model_type == "CPX":
        opt_model = cpx.Model(name="MIP Model")
        var_type = opt_model.integer_var
    elif opt_model_type == "PLP":
        opt_model = plp.LpProblem(name="MIP Model")
        var_type = plp.LpInteger
    else:
        print('Incorrect opt_model_type. Must be grb, cpx, or plp.')
else:
    print('Incorrect x_vars_type')

### Setup contraints parameters and summations

In [6]:
if opt_model_type == "GRB":
    constraint_var = opt_model.addLConstr
    sum_var = grb.quicksum
    if constraint_type == "leq":
        sense_var = grb.GRB.LESS_EQUAL
    elif constraint_type == 'geq':
        sense_var = grb.GRB.GREATER_EQUAL
    elif constraint_type == 'eq':
        sense_var = grb.GRB.EQUAL
    else:
        print('Check constraint_type')
elif opt_model_type == "CPX":
    constraint_var = opt_model.add_constraint
    sum_var = opt_model.sum
    if constraint_type == "leq":
        sense_var = op.le
    elif constraint_type == 'geq':
        sense_var = op.ge
    elif constraint_type == 'eq':
        sense_var = op.eq
    else:
        print('Check constraint_type')
elif opt_model_type == "PLP":
    constraint_var = opt_model.addConstraint
    sum_var = plp.lpSum
    if constraint_type == "leq":
        sense_var = plp.LpConstraintLE
    elif constraint_type == 'geq':
        sense_var = plp.LpConstraintGE
    elif constraint_type == 'eq':
        sense_var = plp.LpConstraintEQ
    else:
        print('Check constraint_type')
else:
    print('Check opt_model_type')

### Set up the system we wish to solve 
1. x_vars: Decision variables - what you're trying to solve for
2. Constraints: these consist of the LHS, RHS, and sense (leq, geq, eq)
3. Define Objective: what you're tring to maximize or minimize

In [7]:
if x_vars_type != "Binary":
    if opt_model_type == "GRB":
        x_vars = {(i,j): opt_model.addVar(vtype =var_type,
                                          lb=lower_bound[i,j],
                                          ub=upper_bound[i,j],
                                          name=name_format.format(i,j))
                  for i in set_I for j in set_J}
        constraints = {j: constraint_var(lhs=sum_var(a[i,j]*x_vars[i,j] for i in set_I),
                                            sense=sense_var,
                                            rhs=b[j],
                                            name=constraint_name.format(j))
                      for j in set_J}
        objective = sum_var(x_vars[i,j]*c[i,j] for i in set_I for j in set_J)
    elif opt_model_type == "CPX":
        x_vars = {(i,j): var_type(lb=lower_bound[i,j],
                                  ub=upper_bound[i,j],
                                  name=name_format.format(i,j))
            for i in set_I for j in set_J}
        constraints = {j: constraint_var(ct=sense_var(sum_var(a[i,j]*x_vars[i,j] for i in set_I),b[j]),
                                        ctname=constraint_name.format(j))
                      for j in set_J}
        objective = sum_var(x_vars[i,j]*c[i,j] for i in set_I for j in set_J)
    elif opt_model_type == "PLP":
        x_vars = {(i,j): plp.LpVariable(cat=var_type,
                                 lowBound=lower_bound[i,j],
                                 upBound=upper_bound[i,j],
                                 name=name_format.format(i,j))
                 for i in set_I for j in set_J}
        constraints = {j: constraint_var(plp.LpConstraint(e=sum_var(a[i,j]*x_vars[i,j] for i in set_I),
                                                   sense = sense_var,
                                                   rhs = b[j],
                                                   name=constraint_name.format(j)))
                      for j in set_J}
        objective = sum_var(x_vars[i,j]*c[i,j] for i in set_I for j in set_J)
    else:
        print('Incorrent opt_model_type')
else:
    if opt_model_type == "GRB":
        x_vars = {(i,j): opt_model.addVar(vtype =var_type,
                                          name=name_format.format(i,j))
                  for i in set_I for j in set_J}
        constraints = {j: constraint_var(lhs=sum_var(a[i,j]*x_vars[i,j] for i in set_I),
                                            sense=sense_var,
                                            rhs=b[j],
                                            name=constraint_name.format(j))
                      for j in set_J}
        objective = sum_var(x_vars[i,j]*c[i,j] for i in set_I for j in set_J)
    elif opt_model_type == "CPX":
        x_vars = {(i,j): var_type(name=name_format.format(i,j))
            for i in set_I for j in set_J}
        constraints = {j: constraint_var(ct=sense_var(sum_var(a[i,j]*x_vars[i,j] for i in set_I),b[j]),
                                        ctname=constraint_name.format(j))
                      for j in set_J}
        objective = sum_var(x_vars[i,j]*c[i,j] for i in set_I for j in set_J)
    elif opt_model_type == "PLP":
        x_vars = {(i,j): plp.LpVariable(cat=var_type, name=name_format.format(i,j))
                 for i in set_I for j in set_J}
        constraints = {j: constraint_var(plp.LpConstraint(e=sum_var(a[i,j]*x_vars[i,j] for i in set_I),
                                                   sense = sense_var,
                                                   rhs = b[j],
                                                   name=constraint_name.format(j)))
                      for j in set_J}
        objective = sum_var(x_vars[i,j]*c[i,j] for i in set_I for j in set_J)
    else:
        print('Incorrent opt_model_type')

### Add min/max condition

In [8]:
if objective_type == 'max':
    if opt_model_type == "GRB":
        opt_model.ModelSense = grb.GRB.MAXIMIZE
        opt_model.setObjective(objective) #defined in preivous cell
    elif opt_model_type == "CPX":
        opt_model.maximize(objective)
    elif opt_model_type == "PLP":
        opt_model.sense=plp.LpMaximize
        opt_model.setObjective(objective) #defined in preivous cell
elif objective_type == 'min':
    if opt_model_type == "GRB":
        opt_model.ModelSense = grb.GRB.MINIMIZE
        opt_model.setObjective(objective) #defined in preivous cell
    elif opt_model_type == "CPX":
        opt_model.maximize(objective)
    elif opt_model_type == "PLP":
        opt_model.sense=plp.LpMinimize
        opt_model.setObjective(objective) #defined in preivous cell
else:
    pint('Check objective_type')



### Solve the model

In [9]:
if opt_model_type == "GRB":
        opt_model.optimize()
elif opt_model_type == "CPX":
        opt_model.solve()
        #opt_model.solve(url="your cplex cloud url", key = "your api key") #cloud cplex solver
elif opt_model_type == "PLP":
        opt_model.solve()
        #opt_model.solve(solver = GLPK_CMD()) #solve using Glpk
else:
    print('check opt_model_type')
#for v in opt_model.variables():
 #   if v.varValue>0:
  #      print(v.name, "=", v.varValue)


### Post-Process the results

In [10]:
opt_df = pd.DataFrame.from_dict(x_vars, orient="index", columns =["variable_object"]) # PANDAS dataframe that holds hte optimal values for each decision variable

opt_df.index = pd.MultiIndex.from_tuples(opt_df.index,names=["column_i", "column_j"])
opt_df.reset_index(inplace=True)
if opt_model_type == "GRB":
    opt_df["variable_object"].apply(lambda item: item.X)
elif opt_model_type == "CPX":
    opt_df["variable_object"].apply(lambda item: item.solution_value)
elif opt_model_type == "PLP":
    opt_df["variable_object"].apply(lambda item: item.varValue)
else:
    print("check opt_model_type")

#opt_df.drop(columns=["variable_object"], inplace=True)
opt_df.to_csv("./optimization_solution.csv")

In [11]:
#for plp
# for v in opt_model.variables():
#     if v.varValue>0:
#         print(v.name, "=", v.varValue)
# for v in opt_model.variables():
#         try: 
#             print(v.name, "=", v.value())
#         except: 
#             print("error couldn't find value")

#For CPLEX
opt_model.print_solution()

objective: 110.167
  x_1_1=1.360
  x_2_1=0.180


In [12]:
#print(constraints)
print(objective)

89.294x_1_1-62.630x_2_1
